In [ ]:
# @title # Installation
!pip -q install git+https://github.com/openai/consistencydecoder.git

!pip -q install numpy open-clip-torch scikit-image>=0.19 timm tomesd torch torchdiffeq torchsde diffusers transformers==4.30.2 accelerate==0.21.0

In [ ]:
# @title # Uploading a picture

import locale
from google.colab import files
import os
import matplotlib.pyplot as plt
from PIL import Image

locale.getpreferredencoding = lambda: "UTF-8"
img_folder = '/content/input' #@param {type:"string"}
!rm -rf {img_folder}
!mkdir -p {img_folder}
%cd {img_folder}
IMAGE = ''

uploaded = files.upload()

for file_name in uploaded.keys():
  IMAGE = os.path.join(img_folder, file_name)

print(f"Original image: {IMAGE}")
plt.figure(figsize=(7, 5))
plt.axis('off')
plt.imshow(Image.open(IMAGE))
plt.show()

In [ ]:
# @title # Processing ConsistencyDecoder

import torch, gc
from diffusers import StableDiffusionPipeline
from consistencydecoder import ConsistencyDecoder, save_image, load_image
import matplotlib.pyplot as plt
from PIL import Image

OUTPUT = '/content/output.png'

gc.collect()
torch.cuda.empty_cache()
pipe = StableDiffusionPipeline.from_pretrained(
    "stablediffusionapi/deliberate3", torch_dtype=torch.float16
)
pipe = pipe.to("cuda")
pipe.vae.cuda()
decoder_consistency = ConsistencyDecoder(device="cuda:0")
image = load_image(IMAGE)

latent = pipe.vae.encode(image.half().cuda()).latent_dist.mean

sample_consistency = decoder_consistency(latent)
save_image(sample_consistency, OUTPUT)

print(f"Processed image: {OUTPUT}")
plt.figure(figsize=(7, 5))
plt.axis('off')
plt.imshow(Image.open(OUTPUT))
plt.show()